Trying to do some hyperparameter tuning

In [1]:
import os
os.chdir('/Users/roberttiu/Desktop/RecSys_Course_AT_PoliMi-master')

from Evaluation.Evaluator import EvaluatorHoldout
import numpy
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
#from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.load_ICM import load_ICM 
from Utils.load_URM import load_URM
from scipy import sparse as sps
import pandas as pd
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization
import json
# import tables

Basics-1-Introduction-to-Python-Programming.ipynb
Basics-2-Numpy.ipynb
Basics-3-Scipy.ipynb
CythonCompiler
Cython_examples
Data_manager
Evaluation
HyperparameterTuning
Notebooks_utils
Practice 02 - How to build a recommender.ipynb
Practice 03 - Content Based Filtering.ipynb
Practice 04 - Collaborative Filtering.ipynb
Practice 05 - Recap - Building an ItemKNN from scratch.ipynb
Practice 06 - SLIM MSE with Gradient Descent and Cython.ipynb
Practice 07 - MF FunkSVD with Cython.ipynb
Practice 08 - MF IALS.ipynb
Practice 08b - MF PureSVD NMF.ipynb
Practice 09 - BPR for SLIM and MF.ipynb
Practice 09 - Hyperparameter optimization with Skopt.ipynb
Practice 11 - MF with PyTorch.ipynb
Practice 12 - Hybrid recommenders.ipynb
Practice 13 - Collaborative Boosted FW.ipynb
Practice 14 - Hybrid model with LightFM on ML.ipynb
Practice extra - Cython for KNN similarity.ipynb
Practice extra 09-old - SLIM BPR and SLIM EN.ipynb
Practice extra 2 - SGD based Learning for Factorization Machines.ipynb
Practice

In [3]:
def combine(ICM: sps.csr_matrix, URM : sps.csr_matrix):
    return sps.hstack((URM.T, ICM), format='csr')

URM_all = load_URM('new_in/data_train.csv') # CSR

ICM_channel_all = load_ICM('new_in/data_ICM_channel.csv') # COO
ICM_event_all = load_ICM('new_in/data_ICM_event.csv')
ICM_genre_all = load_ICM('new_in/data_ICM_genre.csv')
ICM_subgenre_all = load_ICM('new_in/data_ICM_subgenre.csv') 

ICM_all = sps.hstack((ICM_channel_all, ICM_event_all, ICM_genre_all, ICM_subgenre_all))

print(type(ICM_all))
print(type(URM_all))

URM_ICM_all = combine(ICM=ICM_all, URM=URM_all)




<class 'scipy.sparse.coo.coo_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [4]:
URMs_train = []
URMs_validation = []

for k in range(3):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)


evaluator_validation = K_Fold_Evaluator_MAP(URMs_validation, cutoff_list=[10], verbose=True)


ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))
    
    

recommenders = []



tuning_params = {
    "alpha": (1e-4, 1e-2),
    "l1_ratio": (0, 1),
    "topK": (50, 500),
}

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13642 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [7]:
results = []


def BO_func(alpha, l1_ratio, topK):
    recommenders = []

    for index in range(len(URMs_train)):
        recommenders.append(
            MultiThreadSLIM_SLIMElasticNetRecommender(
                URM_train=ICMs_combined[index].T,
                verbose=False
            )
        )

        recommenders[index].fit(
            alpha=alpha,
            l1_ratio=l1_ratio,
            topK=int(topK),
            workers=6
        )

        recommenders[index].URM_train = URMs_train[index]

    result = evaluator_validation.evaluateRecommender(recommenders)
    results.append(result)
    return sum(result) / len(result)

In [8]:
optimizer = BayesianOptimization(f=BO_func, pbounds=tuning_params, verbose=5, random_state=5)

optimizer.maximize(init_points=10,n_iter=2)

import json

recommenders.append(
    MultiThreadSLIM_SLIMElasticNetRecommender(
        URM_train=ICMs_combined[0].T,
        verbose=True
    )
)

recommenders[0].fit()

with open("logs/FeatureCombined" + recommenders[0].RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)


|   iter    |  target   |   alpha   | l1_ratio  |   topK    |
-------------------------------------------------------------


 21%|████████                              | 3816/18059 [00:39<03:40, 64.52it/s]


 41%|███████████████▋                      | 7432/18059 [01:14<02:04, 85.06it/s]


 63%|███████████████████████▍             | 11464/18059 [01:52<01:06, 99.32it/s]


 85%|███████████████████████████████▍     | 15368/18059 [02:31<00:30, 88.00it/s]


...

 47%|█████████████████▍                   | 8520/18059 [01:25<01:26, 109.85it/s]

 47%|█████████████████▉                    | 8552/18059 [01:25<02:15, 69.99it/s]

 48%|██████████████████                    | 8584/18059 [01:26<02:05, 75.37it/s]

 48%|█████████████████▋                   | 8648/18059 [01:26<01:15, 123.89it/s]

 48%|█████████████████▊                   | 8680/18059 [01:26<01:12, 128.59it/s]

 48%|██████████████████▎                   | 8712/18059 [01:27<01:34, 99.26it/s]

 48%|██████████████████▍                   | 8744/18059 [01:27<01:59, 77.76it/s]

 49%|██████████████████▍                   | 8776/18059 [01:27<01:34, 98.38it/s]

 49%|██████████████████                   | 8840/18059 [01:28<01:25, 108.42it/s]

 49%|██████████████████▏                  | 8872/18059 [01:28<01:14, 122.80it/s]

 49%|██████████████████▋                   | 8904/18059 [01:29<02:08, 71.21it/s]

 50%|██████████████████▎                  | 8968/18059 [01:29<01:23, 108.33it/s]

 50%|███████████

 66%|███████████████████████▊            | 11976/18059 [01:59<00:55, 108.97it/s]

 66%|████████████████████████▌            | 12008/18059 [02:00<01:13, 82.15it/s]

 67%|████████████████████████            | 12040/18059 [02:00<00:58, 102.60it/s]

 67%|████████████████████████▋            | 12072/18059 [02:00<01:04, 92.27it/s]

 67%|████████████████████████▊            | 12104/18059 [02:01<01:02, 95.34it/s]

 67%|████████████████████████▉            | 12168/18059 [02:01<01:04, 90.79it/s]

 68%|████████████████████████▍           | 12232/18059 [02:02<00:45, 128.76it/s]

 68%|█████████████████████████▏           | 12264/18059 [02:02<01:02, 93.43it/s]

 68%|█████████████████████████▏           | 12296/18059 [02:03<01:00, 94.51it/s]

 68%|████████████████████████▌           | 12328/18059 [02:03<00:53, 106.97it/s]

 68%|█████████████████████████▎           | 12360/18059 [02:03<01:00, 93.79it/s]

 69%|████████████████████████▊           | 12424/18059 [02:04<00:52, 107.61it/s]

 69%|███████████

 86%|██████████████████████████████▉     | 15528/18059 [02:35<00:22, 113.94it/s]

 86%|███████████████████████████████▉     | 15560/18059 [02:35<00:25, 99.24it/s]

 86%|███████████████████████████████     | 15592/18059 [02:35<00:22, 109.66it/s]

 87%|████████████████████████████████     | 15624/18059 [02:36<00:31, 77.33it/s]

 87%|███████████████████████████████▎    | 15688/18059 [02:37<00:23, 102.23it/s]

 87%|███████████████████████████████▎    | 15720/18059 [02:37<00:22, 102.18it/s]

 87%|████████████████████████████████▎    | 15752/18059 [02:38<00:31, 73.32it/s]

 87%|████████████████████████████████▎    | 15784/18059 [02:38<00:24, 91.61it/s]

 88%|████████████████████████████████▍    | 15816/18059 [02:38<00:25, 87.00it/s]

 88%|████████████████████████████████▍    | 15848/18059 [02:38<00:23, 94.24it/s]

 88%|████████████████████████████████▋    | 15944/18059 [02:39<00:22, 93.50it/s]

 88%|███████████████████████████████▊    | 15976/18059 [02:40<00:20, 102.96it/s]

 89%|███████████

EvaluatorHoldout: Processed 13643 (100.0%) in 6.67 sec. Users per second: 2046
***** Result dict ******
       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
cutoff                                                                      
10      0.331928                 0.333455  0.056889  0.199537    0.200237   

             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
cutoff                                                        ...   
10      0.606816  0.352662  0.097131  0.943781      1.083909  ...   

       COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
cutoff                                                             
10                  0.009857      0.999487              0.943297   

       DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
cutoff                                                             
10           0.002867        6.102988                   0.980576   

       RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTR

KeyError: 10